In [95]:
import json
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from time import localtime, strftime

In [96]:
with open("./data_sno_test_set.txt", "r") as f:
    predict_stns = f.read().split()
    predict_stns.sort()
print("predict_stns:", predict_stns)

predict_stns: ['500101001', '500101002', '500101003', '500101004', '500101005', '500101006', '500101007', '500101008', '500101009', '500101010', '500101013', '500101014', '500101015', '500101018', '500101019', '500101020', '500101021', '500101022', '500101023', '500101024', '500101025', '500101026', '500101027', '500101028', '500101029', '500101030', '500101031', '500101032', '500101033', '500101034', '500101035', '500101036', '500101037', '500101038', '500101039', '500101040', '500101041', '500101042', '500101091', '500101092', '500101093', '500101094', '500101114', '500101115', '500101123', '500101166', '500101175', '500101176', '500101181', '500101184', '500101185', '500101188', '500101189', '500101190', '500101191', '500101193', '500101199', '500101209', '500101216', '500101219', '500105066', '500106002', '500106003', '500106004', '500119043', '500119044', '500119045', '500119046', '500119047', '500119048', '500119049', '500119050', '500119051', '500119052', '500119053', '500119054

In [97]:
def Err_func(b_predict, b_truth, total):
    return 3 * abs(b_predict - b_truth) / total * ( abs((3 * b_truth - total)/(3 *total)) + abs((3 * b_truth - 2 * total)/(3 * total)) )

In [98]:
def val(predic_y, y, total):
  err = 0
  for i in range(len(y)):
    err += Err_func(float(predic_y[i]), y[i], total)

  return err / len(y)

In [99]:
def Load_stn_tot():
  with open("./data_stn_tot.json", 'r') as f:
    stn_tot = json.load(f)
  return dict(stn_tot)

In [100]:
# for released days
def Load_datelist(mode):
    '''
    mode`: should be a `str` in `['train', 'val', 'test', 'release']`\\
    '''
    mode_list = ['train', 'val', 'test', 'release']
    if mode in mode_list:
        with open(f"data_datelist_{mode}.txt", 'r') as f:
            datelist = f.read().split() 
        datelist.sort()
        return datelist
    else:
        raise Exception(f'wrong mode, mode should be a `str` in {mode_list}')

In [101]:
def read_data(stn, mode):
  data = []
  x = []
  y = []
  path = f"./data/{stn}_{mode}.txt"

  with open(path, 'r') as f:
    for line in f.readlines():
      tmp_list = line.split()
      for i in range(len(tmp_list)):
        if i not in [0, 1, 2]:
          tmp_list[i] = float(tmp_list[i])
      # if tmp_list[1] in week_list:
      min = int(tmp_list[2][:2]) * 60 + int(tmp_list[2][3:])
      weekday = int(tmp_list[4])
      all_min = 1440 * (weekday - 1) + min
      if tmp_list[4] == 5 and min > 1440 - 60 * 6:
        tmp_list[5] = 0

      def sin_a(a, b):
        return np.sin(a * 2 * np.pi / b)
      def cos_a(a, b):
        return np.cos(a * 2 * np.pi / b)
      k = 1440
      x_input = []
      rain = float(tmp_list[7])
      # for i in range(1, 7):
      #   x_input.append(sin_a(min, k * i))
      #   x_input.append(cos_a(min, k * i))
      x_input = [sin_a(min, k), cos_a(min, k), sin_a(all_min, k * 7), cos_a(all_min, k * 7)] + tmp_list[5:8]
      y_label = int(tmp_list[8])
    
      
      frac = y_label/ tmp_list[3]
      # if mode != "train":
      #   x.append(list(x_input))
      #   y.append(y_label)
      # elif mode == 'train':
      #   N = 5
      #   for i in range(int(N * 3 * ( abs(frac - 1/3) + abs(frac - 2/3))) - (N - 1)):
      #     x.append(list(x_input))
      #     y.append(y_label)
      #     if weekday in [6, 7]:
      #       for i in range(2):
      #         x.append(list(x_input))
      #         y.append(y_label)
      x.append(list(x_input))
      y.append(y_label)
          
      data.append(tmp_list)
      
  # print("data:", data)
  # print("x_train:", x)
  # print("y_train:", y)
  data = np.array(data)
  x = np.array(x)
  y = np.array(y)
  scaler = StandardScaler()
  # x = scaler.fit_transform(x)
  # y = scaler.transform(y)
  return data, x, y

In [102]:
def run(num_leaves, learning_rate, feature_fraction, num_round):
  all_val_err = 0
  all_train_err = 0
  count = 0
  stn_tot = Load_stn_tot()
  val_json = {}

  predict_file = open(f"./lightGBM_predict_{strftime('%m%d-%H-%M-%S', localtime())}.csv", 'w')
  predict_file.write("id,sbi\n")

  predict_log_file = open(f"./lightGBM_predict_log_{strftime('%m%d-%H-%M-%S', localtime())}.txt", "w")
  print(f"date_val: {Load_datelist('val')}")
  print(f"date_val: {Load_datelist('val')}", file=predict_log_file)
  print(f"date_train: {Load_datelist('train')}")
  print(f"date_train: {Load_datelist('train')}", file=predict_log_file)
  print(f"date_train: {Load_datelist('test')}")
  print(f"date_train: {Load_datelist('test')}", file=predict_log_file)

  # predict_stns = ['500101001']
  for stn in predict_stns:
    count += 1
    data_train, x_train, y_train = read_data(stn, 'train')
    data_val, x_val, y_val = read_data(stn, 'val')
    data_test, x_test, y_test = read_data(stn, 'test')

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data = lgb.Dataset(x_val, label=y_val, reference=train_data)
    params = {
      'objective': 'regression',
      'metric': 'mse',
      'boosting_type': 'gbdt',
      'num_leaves': num_leaves,
      'learning_rate': learning_rate,
      'feature_fraction': feature_fraction,
      'verbose' : -1,
    }

    # Train the model
    
    model = lgb.train(params, train_data, num_round, valid_sets=[val_data])
    pre_train = model.predict(x_train)

    pre_val = model.predict(x_val)
    train_err = val(pre_train, y_train, int(stn_tot[stn]))
    val_err = val(pre_val, y_val, int(stn_tot[stn]))
    all_train_err += train_err
    all_val_err += val_err
    
    log_info = f"{stn}: train: {train_err:<19}, \033[34mval: {val_err:<19}\033[0m, avg_train: {all_train_err / count:<19}, avg_val: {all_val_err/count:<19}"
    print(log_info, file=predict_log_file)
    print(log_info)
    
    val_json[stn] = {}
    val_json[stn]['val'] = val_err
    val_json[stn]['train'] = train_err
    
    
    # plt.figure(figsize=(80, 5), dpi=300)
    # plt.plot(pre_train, label='pre_train', color='b')
    # plt.plot(y_train, label='truth', color='darkorange')
    # plt.legend()
    # plt.show()
    
    # plt.figure(figsize=(80, 5), dpi=300)
    # plt.plot(pre_val, label='pre_train', color='b')
    # plt.plot(y_val, label='truth', color='darkorange')
    # plt.legend()
    # plt.show()
    # if all_val_err/count > 0.38:
    #   return all_val_err/count
    pre_test = model.predict(x_test)
    for i in range(len(data_test)):
      if data_test[i][2][3:] in ["00", "20", "40"]:
        pre_test[i] = pre_test[i] if pre_test[i] > 0 else 0
        id = f'{data_test[i][1]}_{int(data_test[i][0])}_{data_test[i][2]}'
        predict_file.write(f"{id},{pre_test[i]}\n")
        
  log_info = f"final: train: {all_train_err / len(predict_stns)}, val: {all_val_err / len(predict_stns)}"
  print(log_info, file=predict_log_file)
  print(log_info)

  predict_file.close()
  predict_log_file.close()
  with open("./val_lightGBM.json", 'w') as f: 
    json.dump(val_json, f, indent=2)
  return all_val_err / len(predict_stns)

In [103]:
# a = []
# b = []

# for num_leaves in [16, 31, 64]:
#     for feature_fraction in [0.5, 0.7, 0.9]:
#         for learning_rate in [0.03, 0.05, 0.07, 0.09]:
#             for num_round in [10, 25, 50, 75]:
#                 print(num_leaves, feature_fraction, learning_rate, num_round)
#                 a.append(run(num_leaves, learning_rate, feature_fraction, num_round))
#                 b.append((num_leaves, feature_fraction, learning_rate, num_round))

In [104]:
run(num_leaves=31, learning_rate=0.05, feature_fraction=0.9, num_round=100)

date_val: ['20231123', '20231124', '20231125', '20231126', '20231127', '20231128', '20231129']
date_train: ['20231002', '20231003', '20231004', '20231005', '20231006', '20231007', '20231008', '20231009', '20231010', '20231011', '20231016', '20231017', '20231018', '20231019', '20231020', '20231025', '20231026', '20231027', '20231028', '20231029', '20231030', '20231031', '20231101', '20231102', '20231103', '20231104', '20231105', '20231106', '20231107', '20231108', '20231109', '20231110', '20231111', '20231112', '20231113', '20231114', '20231115', '20231116', '20231117', '20231118', '20231119', '20231120', '20231121', '20231122', '20231201', '20231202', '20231203', '20231204', '20231205', '20231206', '20231207', '20231208', '20231209', '20231210', '20231211', '20231212', '20231213']
date_train: ['20231021', '20231022', '20231023', '20231024', '20231218', '20231219', '20231220', '20231221', '20231222', '20231223', '20231224']


500101001: train: 0.22553751073578746, val: 0.27628344005893585, avg_train: 0.22553751073578746, avg_val: 0.27628344005893585
500101002: train: 0.21884808109809037, val: 0.2452154427658377 , avg_train: 0.2221927959169389 , avg_val: 0.26074944141238676
500101003: train: 0.3502465321037816 , val: 0.395156934227986  , avg_train: 0.2648773746458865 , avg_val: 0.3055519390175865 
500101004: train: 0.32946907586364105, val: 0.4720350178041393 , avg_train: 0.2810252999503251 , avg_val: 0.3471727087142247 
500101005: train: 0.26383585712156715, val: 0.3183261413467233 , avg_train: 0.2775874113845735 , avg_val: 0.3414033952407244 
500101006: train: 0.2733294391541744 , val: 0.3833709487946007 , avg_train: 0.27687774934617365, avg_val: 0.3483979874997038 
500101007: train: 0.26144461573356376, val: 0.35595938091135104, avg_train: 0.2746730159729437 , avg_val: 0.3494781865585105 
500101008: train: 0.44590286481962643, val: 0.5068690039413482 , avg_train: 0.296076747078779  , avg_val: 0.3691520387

0.31055661631091996